<a href="https://colab.research.google.com/github/ezzy4me/youtube_comment_tm/blob/main/step_1__youtube_comment_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Crawling youtube data
We first obtain video information for each youtuber and extract video IDs from a given YouTube video URL<sup>[1]</sup>.

We have defined a function to get comments and descriptions of YouTube videos using that video ID.

Collect information, including comments on YouTube videos within a given DataFrame, and check the title and URL of the video in question separately.

**Please check the code for detailed annotation.**

[1]: https://www.youtube.com/watch?v=GcjdHWVo3gA

In [ ]:
# Environment setting
import pandas as pd
from tqdm.notebook import tqdm
from pytube import YouTube
from googleapiclient.discovery import build

# insert own your Api keys
# If there is more than one key, you can put it in a below list.
api_keys = ['example key',
            'example key2']

In [ ]:
# read xlsx file containing url for each youtube video
def read_xlsx_file(xlsx_path):
    df = pd.read_excel(xlsx_path)
    df.columns = ['title', 'url']
    return df

In [ ]:
# load xlsx fiel
rusty_df = read_xlsx_file('./rusty_df.xlsx')
nomad_df = read_xlsx_file('./nomad_df.xlsx')
harun_df = read_xlsx_file('./harun_df.xlsx')
stephie_df = read_xlsx_file('./stephie_df.xlsx')
developed_df = read_xlsx_file('./developed_df.xlsx')

# check the number of youtube video for each youtuber
print(len(rusty_df))  # 9034
print(len(nomad_df))  # 2343
print(len(harun_df))  # 646
print(len(stephie_df))  # 382
print(len(developed_df))  # 750

9034
2343
646
382
750


In [ ]:
# cross check!
rusty_df.head(3)

,title,url
0,HERON LAKE...BOAT RAMP AND VIEW OF LAKE,https://www.youtube.com/watch?v=2la4RlKh00c
1,"COMMENT CALL!!! July 26, 2023",https://www.youtube.com/watch?v=tHct4f30U7Y
2,DRIVING THROUGH SANTE FE NEW MEXICO,https://www.youtube.com/watch?v=hZ5nh9uty2E


## a. definition for crawling using url

In [ ]:
# YouTube url has 11 digits of video from the back.
# Extracts and returns the last 11 characters of the URL, which is the YouTube video ID.
def get_videoid(url):
    return url[-11:]

# get video description
def get_description(video_id):
    url = f'https://www.youtube.com/watch?v={video_id}'  # Constructs the YouTube video URL using the provided video ID.
    yt = YouTube(url)  # Initializes the YouTube video object.
    return yt.description  # Returns the video description.

# get a list of comment texts
def get_comments(video_id, api_key):
    comments = list()  # Initialize an empty list to store comments.
    api_obj = build('youtube', 'v3', developerKey=api_key)  # Sets up the YouTube API connection.
    response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()  # Fetches the first 100 comments for the video.

    while response:  # Continues as long as there are comments to fetch.
        for item in response['items']:  # Iterates over each comment thread.
            comment = item['snippet']['topLevelComment']['snippet']  # Extracts the top-level comment.
            comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])  # Appends the comment details to the list.

            if item['snippet']['totalReplyCount'] > 0:  # Checks if there are any replies to the top-level comment.
                for reply_item in item['replies']['comments']:  # Iterates over each reply.
                    reply = reply_item['snippet']  # Extracts the reply details.
                    comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])  # Appends the reply details to the list.

        if 'nextPageToken' in response:  # Checks if there's another page of comments.
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()  # Fetches the next 100 comments using the nextPageToken.
        else:
            break  # Breaks out of the loop if there's no nextPageToken.

    return [c[0] for c in comments]  # Returns a list of comment texts.


In [ ]:
# get youtube video titles and urls
def get_infos_from_channel(df, api_key, save_path):
    title_lst, url_lst, comments_lst = [], [], []
    re_title_lst, re_url_lst = [], []  # Lists to store titles and URLs causing exceptions.

    for i in tqdm(range(len(df))):
        try:
            title = df.iloc[i]['title']  # Extract the video title from the current row.
            url = df.iloc[i]['url']  # Extract the video URL from the current row.

            video_id = get_videoid(url)  # Get video ID using the URL.
            comments = get_comments(video_id, api_key)  # Fetch comments using the video ID.

            title_lst.append(title)  # Store the title.
            url_lst.append(url)  # Store the URL.
            comments_lst.append(comments)  # Store the comments.
        except:
            print(url)  # Print URLs causing exceptions.
            re_title_lst.append(title)  # Store title causing exception.
            re_url_lst.append(url)  # Store URL causing exception.

    assert len(title_lst) == len(url_lst) == len(comments_lst)  # Check all main lists have same length.
    assert len(re_title_lst) == len(re_url_lst)  # Check retry lists have same length.

    infos = pd.DataFrame({'title': title_lst, 'url': url_lst, 'comments': comments_lst})  # Form a dataframe with fetched information.
    infos.to_csv(save_path, encoding='utf-8-sig', index=False)  # Save this dataframe as a CSV file.

    re_df = pd.DataFrame({'title': re_title_lst, 'url': re_url_lst})  # Form a dataframe of entries causing exceptions.

    return infos, re_df


## b. crawling with comments for each youtuber's video

In [ ]:
stephie_infos = get_infos_from_channel(stephie_df, api_keys[0], './stephie_infos.csv')
# 'Disable YouTube comments' with both exceptions
# https://www.youtube.com/watch?v=1qY5Z9qqi_I
# https://www.youtube.com/watch?v=Tsz6EGYumuE

  0%|          | 0/382 [00:00<?, ?it/s]

https://www.youtube.com/watch?v=1qY5Z9qqi_I
https://www.youtube.com/watch?v=Tsz6EGYumuE


In [ ]:
harun_infos = get_infos_from_channel(harun_df, api_keys[0], './harun_infos.csv')
# https://www.youtube.com/watch?v=Omv3mLVVQSQ

  0%|          | 0/646 [00:00<?, ?it/s]

https://www.youtube.com/watch?v=Omv3mLVVQSQ


In [ ]:
developed_infos = get_infos_from_channel(developed_df, api_keys[0], './developed_infos.csv')
# 'Disable YouTube comments' with five exceptions
# https://www.youtube.com/watch?v=m0WRjY5nTBY
# https://www.youtube.com/watch?v=TUohjVsULhw
# https://www.youtube.com/watch?v=GQ1RIPIDtzw
# https://www.youtube.com/watch?v=-VpuR-vlCB8
# https://www.youtube.com/watch?v=8iEC2sw-6tI

  0%|          | 0/750 [00:00<?, ?it/s]

https://www.youtube.com/watch?v=m0WRjY5nTBY
https://www.youtube.com/watch?v=TUohjVsULhw
https://www.youtube.com/watch?v=GQ1RIPIDtzw
https://www.youtube.com/watch?v=-VpuR-vlCB8
https://www.youtube.com/watch?v=8iEC2sw-6tI


In [ ]:
nomad_infos, nomad_errorurl_lst = get_infos_from_channel(nomad_df, api_keys[1], './nomad_infos.csv')
# 'Disable YouTube comments' with exceptions

  0%|          | 0/2343 [00:00<?, ?it/s]

https://www.youtube.com/watch?v=15DLIUaU0p8
https://www.youtube.com/watch?v=H1zyDyhLCeE
https://www.youtube.com/watch?v=6RINca1Re2I
https://www.youtube.com/watch?v=wHHYncka7GA
https://www.youtube.com/watch?v=JJx4aeJSHAg
https://www.youtube.com/watch?v=eIUPirohkko
https://www.youtube.com/watch?v=YhKdmi7uZdk
https://www.youtube.com/watch?v=F6_ae0NT3_o
https://www.youtube.com/watch?v=ERcBDxwyo24
https://www.youtube.com/watch?v=xYVbFOvnuR0
https://www.youtube.com/watch?v=nR0tuxp73tY
https://www.youtube.com/watch?v=XRe0tAUMGk8
https://www.youtube.com/watch?v=o9YjM1etVoI
https://www.youtube.com/watch?v=kCjR6wTsrzg
https://www.youtube.com/watch?v=YZ19rdv3T-k
https://www.youtube.com/watch?v=oWgpJt2dZ3s
https://www.youtube.com/watch?v=Z7H7RFBPL9I
https://www.youtube.com/watch?v=KJ5mpgBK19Y
https://www.youtube.com/watch?v=gefF-8Ed0ek
https://www.youtube.com/watch?v=0FllB6SrYqo
https://www.youtube.com/watch?v=dKoemfimb8I
https://www.youtube.com/watch?v=czf4s3Mgjts
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=4ajdLA76M4E
https://www.youtube.com/watch?v=82X9xNkJIVI
https://www.youtube.com/watch?v=aaU4srkNZM8
https://www.youtube.com/watch?v=rg0SnUl1Vt0
https://www.youtube.com/watch?v=K8hWiMXhx1I
https://www.youtube.com/watch?v=U-cb_yTUhz0
https://www.youtube.com/watch?v=9-KDi3n03Xs
https://www.youtube.com/watch?v=AVh7cP_BuYM
https://www.youtube.com/watch?v=wKTj2QMfuXw
https://www.youtube.com/watch?v=Z0tqpZUl4eo
https://www.youtube.com/watch?v=JJS1es9-RPU
https://www.youtube.com/watch?v=-tbQSL0sDqc
https://www.youtube.com/watch?v=T9a6RY7gNOA
https://www.youtube.com/watch?v=ZN7XELuh5aU
https://www.youtube.com/watch?v=xcNVuOC3OYU
https://www.youtube.com/watch?v=6blbGs_Z-88
https://www.youtube.com/watch?v=ycydgC2JyeQ
https://www.youtube.com/watch?v=JX1S7t68fTg
https://www.youtube.com/watch?v=OXRg_fXfjVo
https://www.youtube.com/watch?v=Dr_uCC1KhyI
https://www.youtube.com/watch?v=BtIHJc9Co-4
https://www.youtube.com/watch?v=PZEwD42W7cg
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=jYTUAEPqWvU
https://www.youtube.com/watch?v=Le_i76npD-M
https://www.youtube.com/watch?v=zIhzk2KmzhI
https://www.youtube.com/watch?v=Fd9tmMhHtnM
https://www.youtube.com/watch?v=vjRI2MQEom4
https://www.youtube.com/watch?v=M18GJ_5RwLo
https://www.youtube.com/watch?v=nMcu9VcF4Ls
https://www.youtube.com/watch?v=xJNMyGoY6kI
https://www.youtube.com/watch?v=QOISJvMLyQM
https://www.youtube.com/watch?v=5HxNOxP_3cE
https://www.youtube.com/watch?v=dSXeUnRQtns
https://www.youtube.com/watch?v=L-keSul4bbI
https://www.youtube.com/watch?v=i9vk9LEjiJw
https://www.youtube.com/watch?v=z2RmMMb90EE
https://www.youtube.com/watch?v=GhdePJa1Qno
https://www.youtube.com/watch?v=NGcfR3yn71Y
https://www.youtube.com/watch?v=blnTrBJG6UU
https://www.youtube.com/watch?v=DIoulci-YKY
https://www.youtube.com/watch?v=vLunleoDLZ4
https://www.youtube.com/watch?v=j1sGbPkHcGM
https://www.youtube.com/watch?v=HdiKdpMEhwA
https://www.youtube.com/watch?v=UyuC1qmDo4E
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=C7DwlbTu9ss
https://www.youtube.com/watch?v=AIa9bW1JWUk
https://www.youtube.com/watch?v=bL6_2ZuN3MU
https://www.youtube.com/watch?v=MvK3sYhtOxs
https://www.youtube.com/watch?v=EpaLRHYdIlM
https://www.youtube.com/watch?v=MPg8zEEZ5dQ
https://www.youtube.com/watch?v=dMdg5F5D51o
https://www.youtube.com/watch?v=oW04rSKQKm0
https://www.youtube.com/watch?v=R1DfEFUt33Y
https://www.youtube.com/watch?v=RAufEF67lj4
https://www.youtube.com/watch?v=cWZIjcqFIO4
https://www.youtube.com/watch?v=0Rpy0PImXhw
https://www.youtube.com/watch?v=ppzXKNTKpIk
https://www.youtube.com/watch?v=FDQ-R3gAWa0
https://www.youtube.com/watch?v=aUnRYVh2VoY
https://www.youtube.com/watch?v=gUN0eSMxito
https://www.youtube.com/watch?v=5hc12mExs1U
https://www.youtube.com/watch?v=u4V3Drd1erk
https://www.youtube.com/watch?v=l28g47Gbmnc
https://www.youtube.com/watch?v=o5_4UJpLEQY
https://www.youtube.com/watch?v=njLnBnR6nJk
https://www.youtube.com/watch?v=UiiwkUEq1zs
https://www.youtube.com/watch?v=

...

https://www.youtube.com/watch?v=vUFLFqAndYc
https://www.youtube.com/watch?v=O0j39KCBoK0
https://www.youtube.com/watch?v=BQPPBMCxfW8
https://www.youtube.com/watch?v=8wT4742uUJs
https://www.youtube.com/watch?v=VhtooyP5WoA
https://www.youtube.com/watch?v=YvyT6_ZeEiA
https://www.youtube.com/watch?v=4IuAdA0mm0U
https://www.youtube.com/watch?v=GK0OZms2dkw
https://www.youtube.com/watch?v=g16LQp6fXFo
https://www.youtube.com/watch?v=lsat5RCI__0
https://www.youtube.com/watch?v=1wsVXErw8Lk
https://www.youtube.com/watch?v=6nIDG9ZIr94
https://www.youtube.com/watch?v=r8ZLFkuli-o
https://www.youtube.com/watch?v=NImZkKLcbKQ
https://www.youtube.com/watch?v=XutwFa98FeA
https://www.youtube.com/watch?v=QgQc-c2fKmg
https://www.youtube.com/watch?v=s-Nu1RgQ0Nw
https://www.youtube.com/watch?v=GleUQnXE1l8
https://www.youtube.com/watch?v=USGLxFYWWC0
https://www.youtube.com/watch?v=mpcGshdxQ1c
https://www.youtube.com/watch?v=1-M-eoLzVw4
https://www.youtube.com/watch?v=mlgBLULsgwQ
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=thg9v47DHm8
https://www.youtube.com/watch?v=TmEtpNrM8k8
https://www.youtube.com/watch?v=dqXdFp6rFv4
https://www.youtube.com/watch?v=gC9jo21X24o
https://www.youtube.com/watch?v=g-XEztrTRUs
https://www.youtube.com/watch?v=kw7CZ12xR_w
https://www.youtube.com/watch?v=oSUDnxe187g
https://www.youtube.com/watch?v=GAQ_4tUxbUw
https://www.youtube.com/watch?v=TS64-Bc9Kpg
https://www.youtube.com/watch?v=z_fLDTKJ8vk
https://www.youtube.com/watch?v=QQxTRItdV-Y
https://www.youtube.com/watch?v=GI16gLUJBTA
https://www.youtube.com/watch?v=YcxWtgTKUjY
https://www.youtube.com/watch?v=H4z-HOb-Sm0
https://www.youtube.com/watch?v=4Tzf8W5xLRY
https://www.youtube.com/watch?v=dRH9fMZT4nU
https://www.youtube.com/watch?v=CPK6-DthwHo
https://www.youtube.com/watch?v=JP_Y2SjGG6c
https://www.youtube.com/watch?v=sxRosRh0vTA
https://www.youtube.com/watch?v=qglMzAtBZUw
https://www.youtube.com/watch?v=vtRUa57-v3c
https://www.youtube.com/watch?v=enntM1EUgyM
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=LS0kwSxmH98
https://www.youtube.com/watch?v=S2NWRy8eleU
https://www.youtube.com/watch?v=BP0BvDGQcE8
https://www.youtube.com/watch?v=qPKCs2bdAsY
https://www.youtube.com/watch?v=h1X6Oa3Fje4
https://www.youtube.com/watch?v=lsaTTGkVtog
https://www.youtube.com/watch?v=cT7MjUqi2lA
https://www.youtube.com/watch?v=q-upIR7wtWY
https://www.youtube.com/watch?v=ciBrjBk4yDA
https://www.youtube.com/watch?v=2XixzEGXbIU
https://www.youtube.com/watch?v=LxmOPqzpQVw
https://www.youtube.com/watch?v=eOSFAAD9-dU
https://www.youtube.com/watch?v=fk5ZqrJR5DU
https://www.youtube.com/watch?v=7sbP5YvwTI0
https://www.youtube.com/watch?v=vQL4zE_mPdY
https://www.youtube.com/watch?v=oK-85koh9x8
https://www.youtube.com/watch?v=rnEZbuQVjCk
https://www.youtube.com/watch?v=Rx38Dzfpa8M
https://www.youtube.com/watch?v=wLAw8uD0zNQ
https://www.youtube.com/watch?v=_Nm296nCf3E
https://www.youtube.com/watch?v=WaRNy1r9pyI
https://www.youtube.com/watch?v=RLYKAeFRChU
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=ODV4x2Ue80g
https://www.youtube.com/watch?v=6OBk3eUVGSQ
https://www.youtube.com/watch?v=d_v0HahFTpo
https://www.youtube.com/watch?v=c-tL0OxDHjk
https://www.youtube.com/watch?v=xjSDog7VMbA
https://www.youtube.com/watch?v=IDCjv516m0w
https://www.youtube.com/watch?v=v256EuBKRg8
https://www.youtube.com/watch?v=5dZoNp70CiM
https://www.youtube.com/watch?v=wH_RHyFpVcA
https://www.youtube.com/watch?v=lfjqfB4aRjs
https://www.youtube.com/watch?v=_65V5EwJtUs
https://www.youtube.com/watch?v=RY7OHimxCao
https://www.youtube.com/watch?v=rfrP28BRrl4
https://www.youtube.com/watch?v=_PQkYDkPg4c
https://www.youtube.com/watch?v=K4odJ1KxIZA
https://www.youtube.com/watch?v=QJ62pwxaQoY
https://www.youtube.com/watch?v=FgAI5FxuNtQ
https://www.youtube.com/watch?v=UD_v8akvgfk
https://www.youtube.com/watch?v=9_4Dp2ojoJg
https://www.youtube.com/watch?v=TXOu3CU_i5E
https://www.youtube.com/watch?v=G2QgGLB-fQA
https://www.youtube.com/watch?v=J95B6-Sj6Wc
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=gjG2ttfR8gI
https://www.youtube.com/watch?v=Xvl7RcpShro
https://www.youtube.com/watch?v=yo9mGPLJ31U
https://www.youtube.com/watch?v=70rjmMMbnPc
https://www.youtube.com/watch?v=nvGTY3oD0bA
https://www.youtube.com/watch?v=y_thMep1djU
https://www.youtube.com/watch?v=3hiuGe0sokY
https://www.youtube.com/watch?v=QamnmfSE0y8
https://www.youtube.com/watch?v=5VTXNnO7xrg
https://www.youtube.com/watch?v=DsKBFsN4R6k
https://www.youtube.com/watch?v=DkMF0XzLLI0
https://www.youtube.com/watch?v=osjLeE13Xrk
https://www.youtube.com/watch?v=7z-5Ylap7wI
https://www.youtube.com/watch?v=ZOW6vDP43zo
https://www.youtube.com/watch?v=j_Q4IgNrm-8
https://www.youtube.com/watch?v=xyWlpHnRQ8s
https://www.youtube.com/watch?v=YIBiyhZYsXg
https://www.youtube.com/watch?v=v2fj-vgNsvE
https://www.youtube.com/watch?v=oqgA-Yb_zCE
https://www.youtube.com/watch?v=HY1uqC8QKLI
https://www.youtube.com/watch?v=doAQgI3Pk0E
https://www.youtube.com/watch?v=NR6SHK-oNI0
https://www.youtube.com/watch?v=

In [ ]:
# the number of error url list
len(nomad_errorurl_lst)

2084

In [ ]:
rusty_infos, rusty_errorurl_lst = get_infos_from_channel(rusty_df, api_keys[2], './rusty_infos.csv')
# 'Disable YouTube comments' with exceptions

  0%|          | 0/9034 [00:00<?, ?it/s]

https://www.youtube.com/watch?v=kCL-j1f5wVA
https://www.youtube.com/watch?v=OsPdN0RKczQ
https://www.youtube.com/watch?v=RQHodDMMmbA
https://www.youtube.com/watch?v=4V15WNdkOGw


In [ ]:
# check list
rusty_infos